In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn
import requests
import re
import time

In [2]:
#format URL, adding appropriate headers
def URL_format(url):
    http_flag = 0
    www_flag = 0
    new_url = ""
    
    if "http" not in url[:4]:
        http_flag = 1
    if "www" not in url[:11]:
        www_flag = 1

    if http_flag == 1:
        try:
            r = requests.get('https://'+ url, timeout=10)
            new_url = "https://"
        except:
            new_url = "http://"
            
        if www_flag == 1:
            new_url = new_url + "www."
            
        new_url = new_url+url
    else:
        if www_flag == 1:
            if url[:5] == "https":
                new_url = url[:8] + "www." + url[8:]
            elif url[:4] == "http":
                new_url = url[:7] + "www." + url[7:]
        else:
            new_url = url
    time.sleep(1/4)
    return new_url

#Length of URL
def URL_len(url):
    return len(url)

#Length of Hostname
def hostname_len(url):
    #assumes we start with ://www....
    matches = re.findall("://www.([\w\-\.]+)", str(url))
    result = 0
    if (len(matches) >= 1):
        result = len(matches[0])
    return result

#Length of path
def path_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url))[0])
    return match_len

#Length of first directory
def first_dir_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url))[0])
    return match_len
    
#Length of top level domain
def top_level_len(url):
    domains = re.findall("://www.([\w\-\.]+)", str(url))
    if len(domains) != 0:
        tld = domains[0].split('.')
        return len(tld[-1])
    return 0

#Count of '-'
def count_dash(url):
    dashes = 0
    for char in url:
        if char == '-':
            dashes += 1     
    return dashes

#Count of '@'
def count_at(url):
    at = 0
    for char in url:
        if char == '@':
            at += 1     
    return at

#Count of '?'
def count_q(url):
    q = 0
    for char in url:
        if char == '?':
            q += 1     
    return q

#Count of '%'
def count_percent(url):
    percent = 0
    for char in url:
        if char == '%':
            percent += 1     
    return percent

#Count of '.'
def count_dot(url):
    dot = 0
    for char in url:
        if char == '.':
            dot += 1     
    return dot

#Count of '='
def count_equal(url):
    eq = 0
    for char in url:
        if char == '=':
            eq += 1     
    return eq

#Count of ';'
def count_colon(url):
    colon = 0
    for char in url:
        if char == ';':
            colon += 1     
    return colon

#Count of 'www'
def count_www(url):
    www = 0
    www = len(re.findall("www", str(url)))
    return www

#Count of numbers
def count_numbers(url):
    numbers = sum(c.isdigit() for c in url)
    return numbers

#Count of letters
def count_letters(url):
    letters = sum(c.isalpha() for c in url)
    return letters

#Count of directories
def dir_count(url):
    dirs = 0
    for char in url:
        if char == '/':
            dirs = dirs + 1
    #assumes that it starts with ://www....
    return dirs - 2

#Count of single letter directories
def single_letter_dir(url):
    sldir = 0
    sldir = len(re.findall("\/[\w]\/", str(url)))
    return sldir
                
#Count of queries
def query_count(url):
    count = 0
    count = len(re.findall("\?\w+(&?\w+)*", str(url)))
    return count

#Ratio of uppercase to lowercase letters
def ratio_upper_lower(url):
    upperSum = 0
    lowerSum = 0
    for char in url:
        if char.isupper():
            upperSum = upperSum + 1
        if char.islower():
            lowerSum = lowerSum + 1
    # if url has 0 of one, this will return 0
    if lowerSum == 0:
        return upperSum
    return upperSum / lowerSum

#IP vs not
def is_ip(url):
    if len(re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", str(url))) > 0:
        return 1
    else:
        return 0
    
# Shortened or not
def is_shortened(url):
    shortened = { "bit.ly", "tinyurl", "goo.gl", "ow.ly", "t.co", 
                 "tiny.cc", "bit.do", "shorte.st", "cutt.ly", "clkim"}
    
    if any(shrt in url for shrt in shortened):
        return 1
    else:
        return 0

#HTTP vs. HTTPS
#Returns 1 if HTTPS, 0 if HTTP
def is_https(url):
    if url[:5] == 'https':
        return 1
    elif url[:4] == 'http':
        return 0


In [4]:
#Read csv data using pandas
majestic_million=pd.read_csv('../data/majestic_million.csv')
phishtank=pd.read_csv('../data/phistank_verified_online_data.csv')

#Only take the first 10,000 values of the majestic million values
majestic_million=majestic_million.head(10000)

#Rows are store in res[] after being processed
res=[]

malicious=0
benign=1


In [ ]:

for index,row in majestic_million.iterrows():
    #print(row.IDN_Domain)
    url = URL_format(row.IDN_Domain)
    print(index)
    print(url)

    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,benign,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
  
result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result_majestic = pd.DataFrame(res,columns=result_column_names)
result_majestic.to_csv('../data/majestic_out.csv', sep=',',index=False)

0
https://www.google.com
1
https://www.facebook.com
2
https://www.youtube.com
3
https://www.twitter.com
4
https://www.instagram.com
5
https://www.linkedin.com
6
https://www.microsoft.com
7
https://www.apple.com
8
https://www.wikipedia.org
9
https://www.googletagmanager.com
10
http://www.youtu.be
11
https://www.en.wikipedia.org
12
https://www.plus.google.com
13
https://www.pinterest.com
14
https://www.play.google.com
15
https://www.vimeo.com


In [4]:
res=[]
for index,row in phishtank.iterrows():
    #print("original: " + row.url)
    url = row.url
    print(index)
    print(url)
    
    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,malicious,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result = pd.DataFrame(res,columns=result_column_names)
result.to_csv('../data/phistank_out.csv', sep=',',index=False)

0
http://www.mysqleg.secure.runescape.com-gf.ru/
1
https://www.mysqleg.secure.runescape.com-gf.ru/
2
http://www.emaig.secure.runescape.com-gf.ru/
3
http://emaig.secure.runescape.com-gf.ru/
4
http://mysqleg.secure.runescape.com-gf.ru/
5
https://mysqleg.secure.runescape.com-gf.ru/
6
https://www.emaig.secure.runescape.com-gf.ru/
7
http://www.mww.48tees.runescape.com-gf.ru/
8
https://www.mww.48tees.runescape.com-gf.ru/
9
https://emaig.secure.runescape.com-gf.ru/
10
https://mww.48tees.runescape.com-gf.ru/
11
http://mww.48tees.runescape.com-gf.ru/
12
https://olx-pl.safeorder.cyou/1623165790744
13
http://olx.pl-myitem.pw/cash96937365
14
https://olx.pl-myitem.pw/cash96937365
15
http://www.wiitzar.com/cms/MX10/index.html
16
https://rakuten.jp.mnhcrf.shop/
17
http://rakuten.jp.mnhcrf.shop/
18
https://services.runescape.com-vzla.ru/m=weblogin/loginform874,732,675,24156382,2167
19
https://rakuten.jp.snjcrc.shop/
20
http://rakuten.jp.snjcrc.shop/
21
https://elimed.com.ec/wp-includes/i.html#lauraran

http://amazon.stswin520j.top/signim/?openid_pape_max_auth_age=http:/specs.openid.net/auth/2.0/identifier&amp;openid_return_to=https:/www.amazon.co.jp/?ref_=nav_em_hd_re_signin&amp;openid.identity=_select&amp;openid.assoc_handle=jpflex&amp;openid.mode=checkid_setup&amp;key=a@b.c&amp;openid.claimed_id=http:/specs.openid.net/auth/2.0/identifier_select&amp;openid.ns=http:/specs.openid.net/auth/2.0&amp;ref_=nav_em_hd_clc_signin
619
http://cheks2021.000webhostapp.com/
620
http://www.doors.lk/ctct/
621
https://adelfasys.com/config/log/in/82cadb0649a3af4968404c9f6031b233/
622
http://grupwahotnew.duckdns.org/
623
http://lnsteqram.beget.tech/
624
http://lnsteqrams.beget.tech/
625
http://pemersatubangsa-indo.duckdns.org/
626
http://vrifikasionnpaqessfbb.000webhostapp.com/
627
https://www.zononm.co.u2tywg.cn/mobile/
628
http://gruptante-novia.duckdns.org/
629
http://chatwhatsappgroups00044.duckdns.org/
630
http://guddutravelworld.com/ama/7e2ef
631
http://whatisnbphgsbn.duckdns.org/
632
https://msd

http://join-chat551.hotbkpviral56.duckdns.org/
1148
http://heyirv.com/Folder
1149
http://heyirv.com/Folder/
1150
http://www.fastel.ph/TO/authorize_client_id:rh5lje2o-thaw-ub0w-ntsw-mfioy2den1sz_wjv9f0dpxknsa1i5mt62h784cgezlqo3byurpojbv7hlu9tq1d30c8gyf6wnzr5eim4aksx24qw90s2b1jlgv5rehi8ta3cfxn7m6ykuopdz?data=ZGVtb0B3aGF0ZXZlci5sb2w=
1151
http://itaucard-atualizacao-cadastral2.azurewebsites.net/
1152
http://verifications-pages.duckdns.org/
1153
http://www.teknozain.com/TO/authorize_client_id:axgyqwpk-pwzi-bg3j-zgmo-wur5ps8cgoxy_q3kbiepg62vx4dm7z9af8ys1wl50nochujrtqz8uw1gikhmlj9xyvte7352cbnf4apodrs60ijnkxpvlab95egsf0ucy8dzhmtoq341r7w62?data=bWFza2VkQG1hc2tlZC5jb20=
1154
http://594cd78d2116.ngrok.io/instagram
1155
http://a0548407.xsph.ru/postale/authentification/colis=947/
1156
https://fbcom-mdzogliao.homebet88.me/connect.html?timelimit=a459553b10064b9e20a16c47e52291ff
1157
http://prayermeetingplayback.us/files/
1158
http://verifications-pages.duckdns.org/verifications-security-pages.php
11

1665
http://supergaca.com/
1666
https://expensereduction.s3.eu-de.cloud-object-storage.appdomain.cloud/1chenner.html
1667
http://nicenecei.geronmonet.cn/?GHVthXseZzERDXftGVYBHUnjIMjnKBjFCxrdXFTCHyjgJNUHgvHHFCdgJHVJFcdRXDftc
1668
http://configurrations-accntconfrim.ml/Login_support.php
1669
http://nicenecei.geronmonet.cn/?GHVth*seZzERDXftGVYBHUnjIMinKBjFC*rdXFTCHyigJNUHgvHFCdgJHVJFcdR*Dftc
1670
https://galsecus-galsiaonlisnes-ports.000webhostapp.com/
1671
https://validerment.000webhostapp.com/
1672
https://mewwwww.000webhostapp.com/
1673
http://winwithtesla.com/
1674
http://winwithtesla.com
1675
https://ee.review-newbilling.com/
1676
http://dfcc.termcupboard.link/apmix
1677
http://nicenecei.geronmonet.cn/?GHVthXsseZzERDXftGVYBHUnjIMjnKBjFCxrdXFTCHyjgJNUHgvHFCdgJHVJFcdRXDftc
1678
http://www.cutt.ly/onhME0y/
1679
http://gabung-grupbokep2021.duckdns.org/
1680
http://107.172.140.104/
1681
http://107.172.140.104/pc/index.php?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.smbc.

http://www_jacss_co_jp.2rple8.rest/stat/stat.php
2238
https://www_jacss_co_jp.2rple8.rest/stat/stat.php
2239
https://www_jacss_co_jp.2rple8.rest/
2240
http://accountliveout.freecluster.eu/
2241
http://nicenecei.geronmonet.cn/?GHVthX$eZzERDXftGVYBHUni1MjnKBiFC*rd*FTCHyjgINUHgvHFCdgJHVIFcdR*Dftc
2242
http://sig0n04.mipropia.com/index.html?i=1
2243
http://whitelistwallet.com
2244
https://whitelistwallet.com/
2245
https://www_jacss_co_jp.4et6d.rest/
2246
http://amazon.litingzxw.cn/signin/?openid_pape_max_auth_age=http:/specs.openid.net/auth/2.0/identifier&amp;openid_return_to=https:/www.amazon.co.jp/?ref_=nav_em_hd_re_signin&amp;openid.identity=_select&amp;openid.assoc_handle=jpflex&amp;openid.mode=checkid_setup&amp;key=a@b.c&amp;openid.claimed_id=http:/specs.openid.net/auth/2.0/identifier_select&amp;openid.ns=http:/specs.openid.net/auth/2.0&amp;ref_=nav_em_hd_clc_signin
2247
http://accounts-autoscout24.de/login/authentication.html
2248
http://accounts-autoscout24.de/login/twofactorauthent

...

https://machinta.com/
8658
http://paypal.com.service.id999.sorttheweb.com/paypal/a5a006b7e9cd5dd226cbee369112e326MTVjOWQzOTRiMTUzMTk3YWVmNGM0ZTdjNjFiNjEzNDM=
8659
http://paypal.com.service.id999.sorttheweb.com/paypal/a5a006b7e9cd5dd226cbee369112e326MTVjOWQzOTRiMTUzMTk3YWVmNGM0ZTdjNjFiNjEzNDM=/
8660
http://paypal.com.service.id999.sorttheweb.com/paypal/8b19408a1996d326aa823cabbfd0548aNzM1YTlkYmYzNzcyYTY2NzA0NmEyMmYyMDRjNmNjNzM=
8661
http://paypal.com.service.id999.sorttheweb.com/paypal/8b19408a1996d326aa823cabbfd0548aNzM1YTlkYmYzNzcyYTY2NzA0NmEyMmYyMDRjNmNjNzM=/
8662
http://paypal.com.service.id999.sorttheweb.com/paypal/6d19466c2eb82a4abe9cf7cf847308e5MTEwYTcyZjI5NDg0ZTNmY2JlMDZjOTAxMTNkOWU3MzA=/
8663
http://paypal.com.service.id999.sorttheweb.com/paypal/6d19466c2eb82a4abe9cf7cf847308e5MTEwYTcyZjI5NDg0ZTNmY2JlMDZjOTAxMTNkOWU3MzA=
8664
http://paypal.com.service.id999.sorttheweb.com/paypal/4532ea0bb3fc10618de3caad74383a68YTFlMjYxMjQzZmQyMjU1YzhmZWZiNmNhZTUwMWQzYTE=
8665
http://paypal.com.

http://e-leclerc.fr-epicerie.club/
9107
https://betasusturkiye.blogspot.com/
9108
http://173.212.239.242/
9109
https://share.hsforms.com/1OWoQghkBQDO-dfBLtGExeQ4g63f
9110
https://hindmovie.cc/
9111
http://millenniaco-my.sharepoint.com/personal/vgibbs_mhmltd_com/_layouts/15/doc.aspx?sourcedoc={ec69f793-5e80-4ab9-ab2a-66e49de9b392}&amp;action=default&amp;slrid=dca0489f-f03b-b000-9fd0-1e171e182306&amp;originalpath=ahr0chm6ly9tawxszw5uawfjby1tes5zagfyzxbvaw50lmnvbs86bzovzy9wzxjzb25hbc92z2liynnfbwhtbhrkx2nvbs9fcfazywv5qvhybetxexbtnuozchm1suiwaw1pnk90znrqcdrpsnpmqmu0uhlrp3j0aw1lpujzclb1vkrnmtbn&amp;cid=0e1475ff-8901-48a8-aeae-660a9e5b5547
9112
http://keypointtraining-my.sharepoint.com/personal/janeann_keypoint-training_com/_layouts/15/doc.aspx?sourcedoc={9af291d0-87c8-456b-8c74-dddd4a2e5852}&amp;action=default&amp;slrid=5e9d489f-500f-a000-704a-3a9b1d01a72a&amp;originalpath=ahr0chm6ly9rzxlwb2ludhryywluaw5nlw15lnnoyxjlcg9pbnquy29tlzpvoi9nl3blcnnvbmfsl2phbmvhbm5fa2v5cg9pbnqtdhjhaw5pbmdfy29tl0v0

9541
https://onedrive.live.com/view.aspx?resid=357CF596B048E521!68171&amp;ithint=onenote%2c&amp;authkey=!AGRJG_pGeGj6PeQ
9542
http://floorsdirectltd.co.uk/chase/surf4.php
9543
http://floorsdirectltd.co.uk/chase/surf3.php
9544
https://onedrive.live.com/view.aspx?resid=B116B3793040630B!5852&amp;ithint=onenote%2c&amp;authkey=!ALTmjf-4BzB1YGU
9545
https://www.shortlink.net/VERIFY-ATM
9546
https://norcaltc-my.sharepoint.com/personal/acolon_norcaltc_org/_layouts/15/WopiFrame.aspx?guestaccesstoken=YzE2SVUs6Et1YFBzBrRbp0zBlkd6FtBulRUE02WUDHw=&amp;docid=1_1f1c059892dd04acf92bca72fa2b86901&amp;wdFormId={22FA7E1D-2B31-41B8-A33F-0D3A531F6142}&amp;action=formsubmit
9547
https://umzap.com/PROTOCOLO-(197019774904)
9548
https://wu7q5.app.link/?platform=hootsuite
9549
http://inaceinox.pt/js/vendor/security/Update/myaccount/signin
9550
http://inaceinox.pt/image_temp/default/Update/myaccount/signin/
9551
https://norcaltc-my.sharepoint.com/personal/acolon_norcaltc_org/_layouts/15/WopiFrame.aspx?guestacces

9896
http://re-redirection-pp-account-id98763432.blogspot.com/
9897
http://lp2m.umsu.ac.id/wp-includes/widgets/redirect.html
9898
https://re-redirection-acc-id923872635122.blogspot.com/
9899
https://webexert.com
9900
http://encryptdrive.booogle.net/?state={ids:[1zsW2sDoRx1jgl1y1_XDtGIxXpkoMS27F]
9901
https://wilsonvaluation602-my.sharepoint.com/:o:/g/personal/brian_wilsonvaluation_com/EvgZsh2f49NAtJi6I_LnklcBZ46LeDpzwPCKxS6JGi7zMw?e=un6Z
9902
http://www.piandizano.it/sito/wp-content/gallery/css/
9903
http://lp2m.umsu.ac.id/wp-includes/IXR/redirect.html
9904
https://auduboninstitute-my.sharepoint.com/:o:/g/personal/kramsey_auduboninstitute_org/EveSQu6pzsxOjjLjb-YGjEwBwb6DV_wN9eBMuZghm1jKBw?e=bCYSTA
9905
http://web1577.webbox444.server-home.org/E6DF44E1650D6EA3856C4F1B59DACA9A
9906
https://soude-masi.firebaseapp.com/
9907
https://docs.google.com/viewerng/viewer?url=proxy.ge.tt/1/files/8tNpiBy2/0/blob?referrer%3Duser-uR6z6NgiUCtfxJqXnhc2bXYONVSMVcncQdrvc-%26pdf
9908
http://nebojsega.com/k

http://jeffreybcam.net/wp-admin/new/administrator_restore.htm
10264
http://0s.nrxwo2lo.ozvs4y3pnu.cmla.ru/?act=login
10265
http://hetershaven.net/css/03496773043/09572370984380/001/3B762723737B2z.html
10266
http://www.i-m.mx/eupdate/emailaccountupdate/
10267
http://getactive365.com/wp-includes/css/upgrade/web-upgrade/
10268
http://mala-riba.com/view/
10269
http://fpmaam.org/antsafiderana/AMAFI/webmail-email-upgrading-service/MicrosoftEmailUpgrade.html
10270
http://indiankitchenfood.com/menu/status.php
10271
http://souvenirsplace.com/chuakad.php
10272
http://www.ezblox.site/free/jennifer111/helpdesk
10273
http://albel.intnet.mu/File/download_doc.php
10274
http://dongsuh.net/visa/index.html/?/visa.com.br/programa/de/pontos/cadastro.html
10275
http://thescrapescape.com/cccxxxxxxxxxc/doc/doc/processing.html
10276
http://www.thescrapescape.com/cccxxxxxxxxxc/doc/doc/processing.html
10277
http://www.thescrapescape.com/cccxxxxxxxxxc/doc/doc/document.php
10278
http://albel.intnet.mu/File/file_d

In [5]:
#Extra test data
kaggle=pd.read_csv('../data/kaggle_shuff.csv')
kaggle=kaggle.head(10000)

kaggle_rows=[]

malicious=0
benign=1

for index,row in kaggle.iterrows():
    #print(row.url)
    url = row.url
    print(index)
    print(url)
    
    if row.label == 'benign':
        good_or_bad = benign
    else:
        good_or_bad = malicious

    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,good_or_bad,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
  
kaggle_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result_kaggle = pd.DataFrame(res,columns=kaggle_column_names)
result_kaggle.to_csv('../data/kaggle_out.csv', sep=',',index=False)

0
https://www.surftofind.com/jackruby
1
https://www.last.fm/music/Wolf+Parade
2
https://www.angieslist.com/companylist/us/wa/seattle/pac-west-telecomm-inc-reviews-3448865.aspx
3
https://www.realchoice.0catch.com/library/deaths/blnysaline70_72.htm
4
https://www.facebook.com/skylows
5
http://ugma.ge/tmp/mettreajour/redirection.php
6
http://gusevpni.ru/wp-content/languages/crypt/
7
https://www.hoise.com/
8
http://drugpillstore.com/r1qjj1
9
https://www.en.wikipedia.org/wiki/Black_Mafia
10
http://www.vcebhopal.ac.in/blessed/alibaba.com/Login.htm?email=abuse@datsai.com
11
https://www.education.washington.edu/cme/fac.htm
12
https://www.politicalgraveyard.com/bio/asbjornson-ashlay.html
13
http://pistolsmith.com/
14
https://www.popmatters.com/pm/review/deliver-us-from-eva/
15
https://www.historykygovenors.com/williamsylvestertaylor.htm
16
https://www.people.famouswhy.com/rolando_boyce/
17
https://www.drrant.net/
18
http://rofhatcety.ru/gate.php
19
https://cp88.ezyreg.com/~xjxj1038/1/44b637c1a50

596
https://www.imdb.com/name/nm0931220/
597
http://kus.sollers-distribution.pl/doc20161/index.php...
598
http://boefhabiba.000a.biz/lishobo/
599
https://www.mortonwest89.com/
600
https://www.majorityleader.gov/
601
http://sp273.edu.pl/css/index.html
602
https://www.amazon.com/Kansas-City-Sessions-Lester-Young/dp/B0000001NN
603
http://aumentopenis.org/wp-content/plugins/theme-check/misc.php
604
http://cagrisimsigorta.com/modules/gdsagqaf/resource/decoded/
605
https://www.curiousfox.com/history_Wales/montgomeryshire.lasso
606
https://www.uk.linkedin.com/pub/david-usher/15/85B/342
607
http://3designcenter.com/blog/wp-content/plugins/all-in-one-seo-pack/other
608
https://www.facebook.com/group.php?gid=127485404408
609
http://veganvet.net/wp-json/oembed/1.0/embed?url=http%3A%2F%2Fveganvet.net%2Fveterinary-student-advice%2F&format=xml
610
https://www.articles.sun-sentinel.com/2010-11-16/news/fl-katies-kids-hearing-20101116_1_katie-s-kids-learning-center-katie-s-kids-van-driver
611
https://w

1054
https://www.nkcschools.org/ophs/
1055
https://www.menswallet.co.uk/launer-wallets.htm
1056
https://www.myspace.com/israfelworld
1057
https://www.usa-people-search.com/Find-Donna-Pelletier.aspx
1058
https://www.indiex.com/labels/s.html
1059
https://www.imdb.com/name/nm0193104/
1060
https://www.imdb.com/title/tt0080202/
1061
https://www.vibrantmourning.com/wp/
1062
http://eioninfo.com/images/Mailerdaemon/mailer-daemon.html
1063
http://primecaremedicalgh.com/aton/zadacchi.html
1064
https://www.poptower.com/melissa-leo.htm
1065
https://www.lowellchamber.mhsoftware.com/
1066
https://www.youtube.com/watch?v=V3gqLJTu7q8
1067
http://dotartprinting.com/Webmail/outlook.php
1068
https://www.millerclarkanimalhospital.com/
1069
https://www.lodgetorreypines.com/
1070
http://specstarsltd.com/wp-content/upgrade/chase/aae76a79b88ee25394ed962c6400082f
1071
http://cntralpge.clan.su/index.htm
1072
http://classicspeedway.com/ff.exe
1073
http://test2.jessekanclerz.com/m4il/a471a461c0ed02d0097d60b3c5fff

1408
https://www.en.wikipedia.org/wiki/Kolkata_Knight_Riders
1409
http://cnbcmjd34847233312313.short.cm/wutnje/
1410
http://dronetech.no/874ghv3
1411
https://www.mountroyallightblog.com/
1412
https://www.sportspool.com/football/scores/NFL_2001_ll.php
1413
http://mclodesigns.com/edvxmhd
1414
https://www.afi.com/
1415
http://martinezlabalsa.atspace.org/78h8ry
1416
https://www.future-micro.ca/Toon-Boom-Animation/
1417
http://algodonerosdesanluis.com.mx/c/wp-includes/Text/Diff/Engine/usaa/verifycode.html
1418
https://www.itsdemilovato.com/?p=2165
1419
http://fina-mente.com/kitrl2
1420
https://www.kansascity-homes-4-sale.com/
1421
https://www.absoluteastronomy.com/topics/United_States_at_the_1948_Summer_Olympics
1422
https://www.listofcountriesoftheworld.com/nz.html
1423
https://www.hugazombie.com/Zombie-News/Movie-News/Teaser-Poster-for-PARANORMAN
1424
https://www.ca.linkedin.com/pub/kristin-edwards/6/446/75b
1425
https://www.linkedin.com/directory/people/zaratemartinez.html
1426
https://w

https://www.customaniacs.org/forum/custom-labels/104750-marmaduke.html
1806
https://www.northjersey.com/sports/123526139__classic__Showcase.html
1807
http://papapa-1212.zapto.org/1234567890.functions
1808
https://www.cafepress.com/alextoth
1809
https://www.torrentfunk.com/torrent/691278/vandals-fear-of-a-punk-planet.html
1810
https://www.pipl.com/directory/name/Ouimet/Sebastien
1811
https://www.tvnz.co.nz/hunger/show-2301232
1812
https://www.everettruessmovie.com/index2.htm
1813
https://www.yuichimarumaru.blogdrive.com/
1814
http://linsy.eu/config/home/?request_type=LogonHandler&amp;Face=en_US&amp;inav=iNavLnkLog
1815
https://www.antimusic.com/news/11/sep/29John_Fogerty_To_Play_Creedence_Clearwater_Revival_Albums_In_Full_At_Upcoming_Concert.shtml
1816
https://fggc64.wixsite.com/mysite
1817
http://ptnst.co.id/.https/www/outlook.office365.com/w-signin/owa/auth/logon.aspx_url_replace-Current_owa/index.php?umail=b25saW5lcHVyY2hhc2VzQGFuZHJld3NwYXRlbC5jb20N
1818
http://lapdatcamerachatluong

...

http://fdslosart.co.uk/col/serverphp/cp.php?m=login
7982
http://bazurashop.com/images/excel/xcel/69799e84c73ecfb6ac367e52258831...
7983
http://www.isyllabi.com/
7984
https://www.yellowpages.com/uniontown-pa/wmbs-radio-590
7985
http://altts.org/temp/drop/08e7ce760b48e8fc4322a8d475061634/login.php?cmd=login_submit&id=cc523381d0254cee8c0d1e996a6c9624cc523381d0254cee8c0d1e996a6c9624&session=cc523381d0254cee8c0d1e996a6c9624cc523381d0254cee8c0d1e996a6c9624
7986
https://www.nbebasketball.com/w3/2011-0305/2011-big-east-tournament-schedule-tickets/
7987
https://www.houseofnames.com/blodgett-coat-of-arms
7988
https://www.genforum.genealogy.com/richards/page17.html
7989
https://www.usa-people-search.com/Find-Steven-R-Tallman.aspx
7990
https://www.beemp3.com/index.php?q=lehja+leja+re+mehki
7991
https://www.dinerhotline.wordpress.com/2007/12/05/the-diner-a-us-route-40-mainstay-in-indiana-set-to-close/
7992
https://www.clevelandbrowns.com/news/article-1/Browns-announce-coaching-staff/6ada47d8-211c-4

https://www.en.wikipedia.org/wiki/Alias_(TV_series)
8392
http://fasunshi.com/images/icons/youtube/pictures/super.php
8393
https://www.en.wikipedia.org/wiki/Equator_Records_(Canada)
8394
https://www.thefreedictionary.com/Townships
8395
https://www.famousquotearchive.com/QuoteAuthor/Edward_Phelps.aspx
8396
https://www.rottentomatoes.com/celebrity/frank_sinatra/
8397
https://www.benoit.com/
8398
https://www.manta.com/ic/mt6bqkw/ca/construction-beaudin-courville-inc
8399
https://www.ericlecompte.com/
8400
https://www.letrs.indiana.edu/cgi/t/text/text-idx?c=wright2;cc=wright2;view=text;rgn=main;idno=Wright2-0773%3C
8401
http://cepica.org.pe/xsc/index.php
8402
http://genius-versand.de/x41fxj6c
8403
http://masinidebatutcuie.ro/downloader/lib/impots.gouv.fr/8543c7bd48942b077f2a4f8fed55f8dc/
8404
https://paypal-assistenzaclienti.it/my-account/impostazioni
8405
https://www.idolator.com/5686082/the-black-eyed-peas-the-beginning-album-sampler
8406
https://www.cbc.ca/sports/soccer/story/2011/04/24/

8845
https://rhbtradesmart.co.id/wp-content/modules/oauth/ase.com/ChasefooL/
8846
https://www.en-gb.facebook.com/family/Goodwin/2
8847
http://www.beatelement.com.au/mgs/es/index.Php.php
8848
http://icodestartups.com/inc/
8849
https://www.tributes.com/show/Brian-John-Watts-92387040
8850
https://www.youtube.com/watch?v=fUEzLByPGr4
8851
https://www.biomedexperts.com/Profile.bme/146188/Jean-Paul_Bourdineaud
8852
http://agregoh.com.br/review/status/contactform.php
8853
https://www.itunes.apple.com/us/app/pandora-radio/id284035177?mt=8
8854
https://www.redxtapes.bandcamp.com/album/red-x-tapes
8855
https://www.watchbootlegmovies.net/possession-2002
8856
https://www.cars.com/dealers/search-results.action?sc.makes=20015&sc.radius=100&sc.zipCode=43452&sc.city=Ottawa%20County&sc.makeName=Ford&sc.state=OH&sc.textPath=true
8857
https://www.tiff.net/filmsandschedules/tiffbelllightbox/2011/3600000158
8858
https://www.veromi.com/Linda-Piche.aspx
8859
https://www.wn.com/Quebecor
8860
https://www.biomed

https://www.gammonish.com/
9192
https://www.123people.com/s/barry+davis
9193
http://dlbjpwt.goog-upp.pw/
9194
http://lymphoedematherapy.com/board
9195
https://www.en.wikipedia.org/wiki/American_Professional_Soccer_League
9196
https://www.noblesoftconsulting.com/
9197
http://wwwinfopagesop032.000webhostapp.com/recovery-answer.html?=10065877425?fb_source=bookmark_apps
9198
https://www.train.spottingworld.com/City_of_St._Louis_%28passenger_train%29
9199
https://www.sfbg.com/39/28/news_immigrants.html
9200
https://www.lightinthebox.com/Chiffon-Column-One-Shoulder-Sweep-Train-Evening-Dress-inspired-by-Sandrine-Bonnaire-at-Venice-Film-Festival_p66884.html
9201
https://www.wiki.answers.com/Q/How_did_Garret_Hobart_die
9202
https://www.manta.com/c/mm42hgt/out-of-our-minds-images
9203
https://smilesresgate.joomla.com/cf1/index.php
9204
https://www.twitter.com/TigerJabouin
9205
http://cicspj.org/redirect.php?j=k
9206
https://www.ca.answers.yahoo.com/question/index?qid=20110816193324AAwWm8S
9207
h

9588
http://moratti-sales.biz/bin/austin/cp.php?letter=login
9589
https://www.youtube.com/watch?v=l-SRt77aDNo
9590
https://www.youtube.com/watch?v=kLVDXdBo9ys
9591
https://www.moviefone.com/celebrity/patricia-ellis/1791481/biography
9592
http://dpntkiegeokwli.biz/php/upload.php
9593
https://www.imdb.com/title/tt1034389/
9594
https://www.wikimapia.org/53837/Douglas-Mental-Health-University-Institute
9595
https://www.huffingtonpost.ca/2011/10/13/ndp-leadership-race-thomas-mulcair_n_1008955.html
9596
http://www.sosdoudou.com/v4/php/home/AtualizacaoModuloSeguranca20176HdZfgfGmR0LdGLrg5cnvq/
9597
http://www.astrosanjog.com/extra/index.htm
9598
https://www.fusionanomaly.net/lennybruce.html
9599
https://www.hurleychiro.com/
9600
https://www.starfool.com/Vanessa_Lengies.html
9601
https://www.sermoncloud.com/jean-claude/
9602
https://www.curehunter.com/public/pubmed19646324.do
9603
https://www.answers.com/topic/mcclymonds
9604
https://www.pacificstandardtime.com/
9605
https://www.jpopmusic.com/